In [1]:
from ase.build import supercells
from ase.io import read
from time import time
from subprocess import run
import importlib as il
import numpy as np
import os

In [2]:
# Check if LD_LIBRARY_PATH is up to date
os.getcwd() in os.environ['LD_LIBRARY_PATH'].split(':')

True

In [3]:
gao = read('geometry.in', 0, 'aims')

In [4]:
# Number of atoms
gao.get_number_of_atoms()

10

In [5]:
# Target number of atoms: 200
n_target = 300
target_size = n_target / gao.get_number_of_atoms()
print(f'Target size: {target_size:.2f}')

Target size: 30.00


In [6]:
# Python implementation
stime = time()
smatrix = supercells.find_optimal_cell_shape(gao.cell, target_size=target_size, target_shape='sc', verbose=1,
                                            lower_limit=-2, upper_limit=2)
print(f'..found in {time()-stime:.3f}s')

target metric (h_target):
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
normalization factor (Q): 0.0672544
idealized transformation matrix:
[[ 1.19377576  1.19377576  0.        ]
 [-4.8236716   4.8236716   0.        ]
 [ 0.29076261  0.29076261  2.60489829]]
closest integer transformation matrix (P_0):
[[ 1  1  0]
 [-5  5  0]
 [ 0  0  3]]
smallest score (|Q P h_p - h_target|_2): 0.283849
optimal transformation matrix (P_opt):
[[ 1  1  0]
 [-5  5  0]
 [ 0  1  3]]
supercell metric:
[[12.4554  0.      0.    ]
 [ 0.     15.4125  0.    ]
 [ 2.0568  1.5412 17.1242]]
determinant of optimal transformation matrix: 30
..found in 23.224s


## CFFI Fortran

In [7]:
from cffi import FFI
ffi = FFI()
lib = ffi.dlopen("libwrapper.so")

In [8]:
ffi.cdef("void c_find_optimal_cell(double cell[], double target_metric[], int target_size, int smatrix[]);")

In [9]:
cell = gao.cell
target_metric = np.eye(3)
smatrix = np.zeros((3,3), dtype=int)
cell, target_metric, target_size, smatrix

(array([[ 6.22768603, -1.54124518,  0.        ],
        [ 6.22768603,  1.54124518,  0.        ],
        [-1.39028711,  0.        ,  5.7080621 ]]), array([[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]), 30.0, array([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]))

In [10]:
# smatrix  = ffi.cast("int*", smatrix.__array_interface__['data'][0])
smatrix  = ffi.cast("int*", smatrix.ctypes.data)

In [11]:
cell  = ffi.cast("double *", cell.ctypes.data)

In [12]:
target_metric  = ffi.cast("double *", target_metric.ctypes.data)

In [13]:
target_size  = int(target_size)
target_size

30

In [14]:
stime = time()
lib.c_find_optimal_cell(cell, target_metric, target_size, smatrix)
print(f'..found in {time()-stime:.3f}s')

..found in 0.171s


In [15]:
list(smatrix[0:9])

[1, 1, 0, -5, 5, 0, 0, 1, 3]

In [16]:
smat = np.array(list(smatrix[0:9])).reshape((3,3))
smat

array([[ 1,  1,  0],
       [-5,  5,  0],
       [ 0,  1,  3]])

In [17]:
np.linalg.det(smat)

29.99999999999999

In [18]:
gao.cell

array([[ 6.22768603, -1.54124518,  0.        ],
       [ 6.22768603,  1.54124518,  0.        ],
       [-1.39028711,  0.        ,  5.7080621 ]])